# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
file_list = get_files('data/song_data')

In [25]:
df = pd.read_json(file_list[-1],lines = True)
df.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,AREVWGE1187B9B890A,-13.442,Noci (BA),-41.9952,Bitter End,282.43546,1,SOFCHDR12AB01866EF,Living Hell,0


In [ ]:
#####

In [5]:
for i in range(len(file_list)):
    df = pd.read_json(file_list[i],lines = True)
    df = df.loc[:, ['song_id', 'title', 'artist_id', 'year', 'duration']].values.tolist()
    cur.execute(song_table_insert, df.pop())
    conn.commit()

In [ ]:
######

## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [19]:
song_data = df.loc[:, ['song_id', 'title', 'artist_id', 'year', 'duration']].values.tolist()
song_data

[['SOFCHDR12AB01866EF', 'Living Hell', 'AREVWGE1187B9B890A', 0, 282.43546]]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [20]:
cur.execute(song_table_insert, song_data.pop())
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [6]:
for i in range(len(file_list)):
    df = pd.read_json(file_list[i],lines = True)
    df = df.loc[:, ['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']].values.tolist()
    cur.execute(artist_table_insert, df.pop())
    conn.commit()

In [ ]:
artist_data = 
artist_data

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [ ]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [7]:
file_list = get_files('data/log_data')

<!-- log_files = 

filepath =  -->

In [5]:
df = pd.read_json(file_list[4],lines = True)
df.head(1)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,matchbox twenty,Logged In,Jayden,F,0,Duffy,177.65832,free,"Seattle-Tacoma-Bellevue, WA",PUT,NextSong,1.540146e+12,846,Argue (LP Version),200,1543109954796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",76


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [ ]:
#####

In [8]:
for i in range(len(file_list)):
    df = pd.read_json(file_list[i],lines = True)
    df = df.loc[df['page'] == 'NextSong', :]
    df['ts_stand'] = pd.to_datetime(df['ts'], unit = 'ms')
    time_data = [df['ts'], df['ts_stand'].dt.hour, df['ts_stand'].dt.day, df['ts_stand'].dt.weekofyear, df['ts_stand'].dt.month, df['ts_stand'].dt.year, df['ts_stand'].dt.weekday]
    column_labels = ['timestamp', 'hour', 'day', 'weekofyear', 'month', 'year', 'weekday']
    time_data_dict = {a : b for a,b in zip(column_labels, time_data)}
    time_df = pd.DataFrame(data = time_data_dict)
    for j, row in time_df.iterrows():
        cur.execute(time_table_insert, list(row))
        conn.commit()

In [ ]:
#####

In [8]:
df = df.loc[df['page'] == 'NextSong', :]
# df.head(1)

In [9]:
# t = pd.to_datetime(df['ts'], unit = 'ms')
df['ts_stand'] = pd.to_datetime(df['ts'], unit = 'ms')
# df.head(1)

In [10]:
time_data = [df['ts'], df['ts_stand'].dt.hour, df['ts_stand'].dt.day, df['ts_stand'].dt.weekofyear, df['ts_stand'].dt.month, df['ts_stand'].dt.year, df['ts_stand'].dt.weekday]
column_labels = ['timestamp', 'hour', 'day', 'weekofyear', 'month', 'year', 'weekday']

In [11]:
time_data_dict = {a : b for a,b in zip(column_labels, time_data)}

In [12]:
time_df = pd.DataFrame(data = time_data_dict)
time_df.head()

,timestamp,hour,day,weekofyear,month,year,weekday
0,1542412944796,0,17,46,11,2018,5
1,1542413200796,0,17,46,11,2018,5
2,1542413451796,0,17,46,11,2018,5
3,1542413675796,0,17,46,11,2018,5
4,1542413859796,0,17,46,11,2018,5


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [9]:
for i in range(len(file_list)):
    df = pd.read_json(file_list[i],lines = True)
    for i, row in time_df.iterrows():
        try:
            cur.execute(time_table_insert, list(row))
            conn.commit()
    #         print(f'{i} of row {row}')
        except psycopg2.Error as e:
            print(e)
    #         break

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [9]:
df.head(1)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId,ts_stand
0,Kenny G,Logged In,Chloe,F,53,Cuevas,256.57424,paid,"San Francisco-Oakland-Hayward, CA",PUT,NextSong,1.540941e+12,648,Everlasting,200,1542412944796,Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20...,49,2018-11-17 00:02:24.796


In [10]:
user_df = df[['userId', 'firstName', 'lastName', 'gender', 'level']].drop_duplicates()

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [9]:
for i in range(len(file_list)):
    df = pd.read_json(file_list[i],lines = True)[['userId', 'firstName', 'lastName', 'gender', 'level']].drop_duplicates()
    for i, row in df.iterrows():
        try:
            cur.execute(user_table_insert, row)
            conn.commit()
    #         print(f'{i} of row {row}')
        except psycopg2.Error as e:
            print(e)

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [16]:
df.shape

(127, 19)

In [20]:
for i in range(len(file_list)):
    df = pd.read_json(file_list[i],lines = True)
    for index, row in df.iterrows():

        # get songid and artistid from song and artist tables
        cur.execute(song_select, (row.song, row.artist, row.length))
        results = cur.fetchone()

        if results:
            songid, artistid = results
            print(results)
        else:
            songid, artistid = None, None

        # insert songplay record
        songplay_data = (
            index,   # f'{songid}_{artistid}', 
            row.ts, 
            row.userId, 
            row.level, 
            songid, 
            artistid, 
            row.sessionId, 
            row.location, 
            row.userAgent
        )
        cur.execute(songplay_table_insert, songplay_data)
        conn.commit()

('SOZCTXZ12AB0182364', 'AR5KOSW1187FB35FF4')


In [11]:
songid

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [3]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.